In [1]:
import pandas as pd
import numpy as np

load_purchase_data = "Resources/purchase_data.json"
purchase_data = pd.read_json(load_purchase_data)

columns = [
    "Age",
    "Gender",
    "Item ID",
    "Item Name",
    "Price",
    "SN"]

purchase_data.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [2]:
#making the "SN" column a data frame
name_data_df = pd.DataFrame(purchase_data["SN"])

#counting the unique players, returning it as a dataframe
total_players = name_data_df.nunique()
total_players = pd.DataFrame({"Total Players": total_players})
total_players
#I couldn't figure out how to get rid of the "SN" and replace it with a 0 just like in the HeroesofPymoli_starter.ipynb file

,Total Players
SN,573


In [3]:
#Pull the column Item Name and count the # of unique items
item_data = purchase_data["Item Name"]
item_count = item_data.nunique()

#Get the avg price, total purchases and total revenue
average_price = purchase_data["Price"].mean()
total_purchases = purchase_data["Price"].count()
total_revenue = purchase_data["Price"].sum()

#create new columns and fill with the data above
purchase_analysis_df = pd.DataFrame([{"Number of Unique Items": item_count, "Average Price": average_price, 
                                      "Number of Purchases": total_purchases,"Total Revenue": total_revenue}])
#make the columns above become a dataframe
purchase_analysis_df = purchase_analysis_df[["Number of Unique Items", "Average Price", 
                                             "Number of Purchases", "Total Revenue"]]
purchase_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,2.931192,780,2286.33


In [4]:
#Group the data by the SN and Gender column. The .size() function returns the unique groups of name/gender (eliminates duplicates).
#then use .reset_index() to restore the grouped columns (makes it easier to count, the .size() is returned in a new column)
#rename the new column using .rename()
gender_groups_one = purchase_data.groupby(["SN","Gender"]).size().reset_index().rename(columns = {0: "Purchases"})

#value count for each gender category
gender_counts = gender_groups_one["Gender"].value_counts()

#the percentage of players each gender makes up
gender_percentages = gender_counts / gender_counts.sum()

#grab gender_counts and gender_percentages from above and put into a df
gender_counts_df = pd.DataFrame({"Total Count":gender_counts, "Percentage of Players": gender_percentages})
gender_counts_df


,Total Count,Percentage of Players
Male,465,0.811518
Female,100,0.174520
Other / Non-Disclosed,8,0.013962


In [5]:
#return gender and price columns as neww data frame
sort_gender = purchase_data[["Gender","Price"]]

#groupby gender and then count 
gender_count = sort_gender.groupby(["Gender"])
gender_count = gender_count.count()
gender_count = pd.DataFrame(gender_count)

#find the unique values of gender
gender_unique = sort_gender["Gender"].unique()

#finding the female gender stats
gender_female = purchase_data.loc[purchase_data["Gender"] == "Female", columns]
female_revenue = gender_female["Price"].sum()
female_average = gender_female["Price"].mean()

#finding the male gender stats
gender_male = purchase_data.loc[purchase_data["Gender"] == "Male", columns]
male_revenue = gender_male["Price"].sum()
male_average = gender_male["Price"].mean()

#finding the other/non-disclosed gender stats
gender_other = purchase_data.loc[purchase_data["Gender"] == "Other / Non-Disclosed", columns]
other_revenue = gender_other["Price"].sum()
other_average = gender_other["Price"].mean()

#Putting the above stats into a dictionary for each gender
gender_revenues = {"Gender":["Female", "Male", "Other / Non-Disclosed"],
                   "Total Purchase Value": [female_revenue, male_revenue, other_revenue],
                   "Average Purchase Price":[female_average, male_average, other_average],
                  "Avg Total Purchase per Person": [female_revenue/100, male_revenue/465, other_revenue/8]}

#Convert the dictionary into a data frame
gender_revenues = pd.DataFrame(gender_revenues, columns=["Gender","Average Purchase Price",
                                                        "Total Purchase Value", "Avg Total Purchase per Person"])
gender_counts = pd.DataFrame(gender_counts)

#merge gender_count and gender_revenues and display
merge_table = pd.merge(gender_count, gender_revenues,on="Gender")
merge_table = merge_table.rename(columns = {"Price": "Purchase Count"})
merge_table


,Gender,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
0,Female,136,2.815515,382.91,3.829100
1,Male,633,2.950521,1867.68,4.016516
2,Other / Non-Disclosed,11,3.249091,35.74,4.467500


In [6]:
#Make bins for each age group
age_bins = [0, 9, 14, 19, 24, 29, 34, 39, 120]
group_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#Count the unique pairs of names and age
age_group = purchase_data.groupby(["SN", "Age"]).size().reset_index().rename(columns = {0:"Occurances"})

#Slice the data and place it into bins
age_group["Age Group"] = pd.cut(age_group["Age"], age_bins, labels=group_labels)
age_group = pd.DataFrame(age_group[["SN", "Age Group"]])

#Re-name columns and group by age group
names_ages = age_group.rename(columns={"SN": "Total Count"})
names_ages = names_ages.groupby("Age Group")

#count the number of plauyers per age group and take the percentage
age_groups_df = pd.DataFrame(names_ages.count())                                
age_groups_df["Percentage of Players"] = (age_groups_df["Total Count"] / int(name_data_df.nunique())) * 100
age_groups_df

,Total Count,Percentage of Players
Age Group,,
<10,19,3.315881
10-14,23,4.013962
15-19,100,17.452007
20-24,259,45.200698
25-29,87,15.183246
30-34,47,8.202443
35-39,27,4.712042
40+,11,1.919721


In [7]:
#Make bins for each age group
age_bins = [0, 9, 14, 19, 24, 29, 34, 39, 120]
group_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#Slice the data and place it into bins
pd.cut(purchase_data["Age"], age_bins, labels=group_labels)

purchase_data["Age Group"] = pd.cut(purchase_data["Age"], age_bins, labels=group_labels)
age_group = purchase_data.groupby("Age Group")

#find purchasse count, avg value, total value, and avg purchase
age_group_sum = age_group["Price"].sum()
age_group = age_group[["Age Group"]].size()
age_group_average = age_group_sum / age_group
avg_total_age = age_group_sum / (age_groups_df["Total Count"])


#turn the functions above into df's
age_group = pd.DataFrame(age_group)
age_group_sum = pd.DataFrame(age_group_sum)
age_group_average = pd.DataFrame(age_group_average)
avg_total_age = pd.DataFrame(avg_total_age)

#merge df's above by age group
age_analysis = pd.merge(age_group, age_group_average, on="Age Group")
age_analysis = pd.merge(age_analysis,age_group_sum,on="Age Group")
age_analysis = pd.merge(age_analysis,avg_total_age,on="Age Group")
age_analysis

#rename columns
age_analysis = age_analysis.rename(columns={"0_x":"Purchase Count", "0_y":"Average Purchase Price",
                                            "Price":"Total Purchase Value", 0:"Avg Total Purchase per Person"})
age_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,28,2.980714,83.46,4.392632
10-14,35,2.770000,96.95,4.215217
15-19,133,2.905414,386.42,3.864200
20-24,336,2.913006,978.77,3.779035
25-29,125,2.962640,370.33,4.256667
30-34,64,3.082031,197.25,4.196809
35-39,42,2.842857,119.40,4.422222
40+,17,3.161765,53.75,4.886364


In [8]:
#find unique pairs of names and hoe much they spent
top_spenders = purchase_data.groupby(["SN", "Price"]).size().reset_index().rename(columns = {0:"Purchases"})

#groupby name, take sum of purchases and average purchase price
name_counts1 = top_spenders.groupby(["SN"])
price_per_name_sum = name_counts1.sum()
price_per_name_mean = name_counts1.sum() / name_counts1.count()

#merge the df's above by name
top_spenders_df = pd.merge(price_per_name_sum,price_per_name_mean,on="SN")

#sort the df and rename the columns
top_spenders_df = top_spenders_df.sort_values(["Price_x"], ascending=False)
top_spenders_df = top_spenders_df.rename(columns = {"Price_x":"Total Purchase Price", "Purchases_x":"Purchase Count",
                                                    "Price_y":"Average Purchase Price"})

top_spenders_df = top_spenders_df[["Purchase Count", "Average Purchase Price","Total Purchase Price"]]
top_spenders_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Price
SN,,,
Undirrala66,5,3.412000,17.06
Saedue76,4,3.390000,13.56
Mindimnya67,4,3.185000,12.74
Haellysu29,3,4.243333,12.73
Eoda93,3,3.860000,11.58


In [9]:
#find unique pairs for Item ID, Item Name and Price
popular_items = purchase_data.groupby(["Item ID", "Item Name", "Price"]).size().reset_index().rename(columns = {0:"Purchases"})

#groupby Item ID and Item Name and printprice and # of purchases
popular_id_sum = popular_items.groupby(["Item ID","Item Name"]).sum()

#Total purchase value 
popular_id_total =(popular_id_sum["Price"] * popular_id_sum["Purchases"])
popular_id_total = pd.DataFrame(popular_id_total)

#merge the two data frames from aboce
popular_items_df = pd.merge(popular_id_sum,popular_id_total,on=["Item ID", "Item Name"])

#rename columns and sort
popular_items_df = popular_items_df.rename(columns = {"Price":"Item Price", "Purchases":"Purchase Count",
                                                      0:"Total Purchase Value"})
popular_items_df = popular_items_df.sort_values(["Purchase Count"], ascending = False)
popular_items_df.head()

,,Item Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",2.35,11,25.85
84,Arcane Gem,2.23,11,24.53
31,Trickster,2.07,9,18.63
175,Woeful Adamantite Claymore,1.24,9,11.16
13,Serenity,1.49,9,13.41


In [10]:
#re-sort the table from above
popular_items_df = popular_items_df.sort_values(["Total Purchase Value"], ascending=False)
popular_items_df.head()

,,Item Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,4.14,9,37.26
115,Spectral Diamond Doomblade,4.25,7,29.75
32,Orenmir,4.95,6,29.70
103,Singed Scalpel,4.87,6,29.22
107,"Splitter, Foe Of Subtlety",3.61,8,28.88
